<a href="https://colab.research.google.com/github/edgasanc85/AzureOpenAI/blob/main/ChatGPT_Reforma_a_la_salud_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatBot basado en el documento de reforma al sistema de salud

In [1]:
%%capture
!pip install langchain langchain-openai pypdf openai chromadb tiktoken

In [2]:
from getpass import getpass
import os
import openai

openai.api_type = "azure"
openai.api_base = getpass('Enter the end point: ')
openai.api_base = openai.api_base
openai.api_version = "2023-07-01-preview"

openai.api_key = getpass('Enter the secret value: ')
os.environ['OPENAI_API_KEY'] = openai.api_key

os.environ["AZURE_OPENAI_ENDPOINT"] = openai.api_base
os.environ["AZURE_OPENAI_API_KEY"] = openai.api_key

Enter the end point: ··········
Enter the secret value: ··········


## Carga de documentos

In [8]:
import requests
from langchain.document_loaders import PyPDFLoader

urls = [
    'https://edgasanc.com/storage/230213-Reforma-salud.pdf'
]

ml_papers = []

for i, url in enumerate(urls):
    response = requests.get(url)
    filename = f'paper{i+1}.pdf'
    with open(filename, 'wb') as f:
        f.write(response.content)
        print(f'Descargado {filename}')

        loader = PyPDFLoader(filename)
        data = loader.load()
        ml_papers.extend(data)

# Utiliza la lista ml_papers para acceder a los elementos de todos los documentos descargados
print('Contenido de ml_papers:')
print()

Descargado paper1.pdf
Contenido de ml_papers:



In [9]:
type(ml_papers), len(ml_papers), ml_papers[6]

(list,
 290,
 Document(page_content='normativas, evaluar el funcionamiento del Sistema de Salud y generar informes periódicos sobre el desenvolvimiento del mismo.  Composición. El Consejo Nacional de Salud estará conformado así:  1. Cinco (5) representantes del Gobierno Nacional: el Ministro de Salud y Protección Social, quien presidirá; el ministro de Hacienda; el Ministro de Trabajo; el Director de Planeación Nacional y el Ministro de Medio Ambiente y Desarrollo Sostenible.  2. Tres (3) representantes de las Secretarías territoriales de Salud. 3. Dos (2) representantes del sector empresarial: uno de las grandes empresas y uno de las pequeñas y medianas empresas, elegidos por sus asociaciones. 4. Dos (2) representantes de las centrales obreras. 5. Dos (2) representantes de las asociaciones campesinas. 6. Siete (7) representantes de las organizaciones de pacientes, uno por cada región sociocultural del país (Central, Pacífica, Atlántica, Eje cafetero, Nororiental, Orinoquia y Amazonia)

## Split de documents

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 1500,
    chunk_overlap= 200, #caracteres que se repiten en el fragmento pasado y siguiente para que tenga continuidad
    length_function= len
    )

documents = text_splitter.split_documents(ml_papers)

In [11]:
len(documents), documents[10]

(784,
 Document(page_content='Sistema de Salud son las de financiamiento, de administración y gestión de salud, de prestación de servicios de salud, de información, de participación social y de inspección, vigilancia y control, definidas en la presente Ley. 21. Entidades Promotoras de Salud: Las entidades promotoras de salud son las entidades públicas, privadas o mixtas que ejercerán actividades de acuerdo a lo que disponga la presente ley y su reglamentación.    CAPÍTULO II.  DETERMINANTES SOCIALES DE LA SALUD', metadata={'source': 'paper1.pdf', 'page': 3}))

## Embeddings e ingesta a base de datos vectorial

In [12]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = AzureOpenAIEmbeddings(
    azure_deployment="demo_embedding-ada",
    openai_api_version="2023-07-01-preview",
)

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings
)

retriever = vectorstore.as_retriever(
    search_kwargs = {"k": 3}
    )

## Modelos de chat y cadenas para consulta de información

In [13]:
from langchain.chains import RetrievalQA
from langchain_openai import AzureChatOpenAI

chat = AzureChatOpenAI(
    openai_api_version="2023-07-01-preview",
    azure_deployment="demo_gpt35",
    temperature=0.0
)

qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff",
    retriever=retriever
)

In [14]:
query = "Existe alguna estimación del costo financiero de la reforma?"
qa_chain.run(query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Sí, se realizaron simulaciones y proyecciones financieras para abordar la sostenibilidad financiera del sistema de salud que se adopta con la reforma. Sin embargo, es importante tener en cuenta que un nuevo sistema de salud que se construye sobre lo construido hereda, desde el punto de vista financiero y con probable impacto fiscal, un conjunto de problemáticas asociadas a los desbalances que se vienen generando a la fecha. Por lo tanto, se efectúa un análisis riguroso de las condiciones de partida del proceso de reforma, evaluando los riesgos de sostenibilidad del mismo y el impacto en la gestión de riesgo en salud. Puedes encontrar más información en el Anexo 3 "La Reforma al Sistema de Salud: un desafío de todos". No se proporciona una cifra específica del costo financiero de la reforma.'

In [15]:
query = "Que va a suceder con las Entidades Promotoras de Salud?"
qa_chain.run(query)

'Las Entidades Promotoras de Salud permanecerán en el Sistema de Salud durante el proceso de transición y evolución hacia el nuevo Sistema. Escindirán progresivamente sus instituciones de prestación de servicios, de acuerdo al plan de implementación que determine el Ministerio de Salud y Protección Social. Durante su permanencia en el periodo de transición o cuando hagan su tránsito a instituciones prestadoras de servicios de salud a través de sus redes propias que funcionaban en el Sistema General de Seguridad Social en Salud, podrán concurrir en la organización y prestación de los servicios de los Centros de Atención Primaria Integral Resolutiva en Salud. Además, podrán acordar con el Ministerio de Salud y Protección social, la ADRES o las entidades territoriales, según corresponda, cuando decidan la suspensión de operaciones y no deseen transitar a la prestación de servicios de salud, la venta de infraestructura de servicios de salud u otras salud, la cual se hará en forma progresiv

In [16]:
query = "Que son los Centros de Atención Primaria Integral Resolutiva en Salud?"
qa_chain.run(query)

'Los Centros de Atención Primaria Integral Resolutiva en Salud son lugares que ofrecen servicios básicos de salud, programas de promoción de la salud, prevención de la enfermedad y referencia de pacientes hacia servicios de mediana o alta complejidad. Estos centros están habilitados para cuidar la salud y prevenir enfermedades en el grupo poblacional de su cobertura y solucionar inmediatamente sus requerimientos de atención básica y solicitudes de referencia y contrarreferencia a las redes de mediana y alta complejidad. En los centros urbanos, cada Centro de Atención Primaria Integral Resolutiva en Salud cubrirá como máximo 25.000 habitantes para garantizar la capacidad de cuidado de la salud y prevención de la enfermedad de dicho grupo poblacional y la solución inmediata de sus requerimientos de atención básica y solicitudes de referencia y contrarreferencia a las redes de mediana y alta complejidad.'

In [17]:
query = "Que dice el texto de reforma acerca de los tipos de vinculación del personal de salud?"
qa_chain.run(query)

'El texto menciona dos tipos de vinculación laboral para los trabajadores estatales de la salud en las Instituciones de Salud del Estado (ISE): los cargos de confianza, que son de libre nombramiento y remoción correspondientes a la conducción, orientación o directrices, y los demás servidores públicos que se rigen por el régimen especial desarrollado en el artículo 73. Además, se garantiza el derecho de permanencia de los servidores y se establece que el sistema de control interno se regirá por la Ley 87 de 1993 y la designación del responsable de la oficina se dará conforme a las Leyes 87 y 1474. Para las Instituciones Privadas y Mixtas del Sector Salud, se establece que las personas vinculadas estarán mediante contrato de trabajo conforme a los preceptos del Código Sustantivo del Trabajo y normas concordantes.'

In [18]:
query = "Como se hara la distribución de recursos en el nuevo sistema?"
qa_chain.run(query)

'En el nuevo sistema de salud, la distribución de recursos será descentralizada y desconcentrada mediante un manejo técnico y transparente de los recursos. La Entidad Administradora de los Recursos del Sistema de Salud, ADRES, será quien administre los recursos públicos de la salud, parafiscales en el orden nacional, tal como indica los artículos del 21 al 33 del proyecto de ley. Los recursos del Fondo Único Público para la Salud a los Fondos Territoriales de Salud serán los mismos del Sistema General de Participaciones (SGP), la equidad y la rentabilidad social, en donde se establecen las normas de destinación y competencias nacionales, regionales y territoriales a través de los artículos del 34 al 52 del proyecto de Ley. La gestión del riesgo financiero estará a cargo del Estado a través de ADRES y la gestión del riesgo en salud en la Atención Primaria Integral en Salud en los territorios. Además, existirán instancias territoriales como el Consejos Regionales de Administración de Seg